In [31]:
import numpy as np
import pandas as pd

### Este notebook auna todo el preprocesado necesario para transformar los datos de la carpeta RAW a PREPARED:

In [32]:
path_in  = "../raw/trabajo6.csv"
path_out = "../partitioned/"

In [44]:
df = pd.read_csv(path_in)

### One Hot Encoding

In [34]:
df = df.join(pd.get_dummies(df['type']))

### Nuevas variables

In [35]:
df['diffOrigen'] = df['newbalanceOrig'] - df['oldbalanceOrg']
df['diffDestino'] = df['newbalanceDest'] - df['oldbalanceDest']

mean_oldOrg  = df['oldbalanceOrg'].mean()
mean_oldDest = df['oldbalanceDest'].mean()

# Para evitar NaN. No afecta la distribución de los datos.
df['cambioOrigen'] = (df['diffOrigen'] + 0.01)/(df['oldbalanceOrg'] + 1e3)
df['cambioDestino'] = (df['diffDestino'] + 0.01)/(df['oldbalanceDest'] + 1e3)

### Particionamos el dataset en 12 partes:

In [36]:
df['partition'] = np.random.randint(12, size=len(df))

In [37]:
df.partition.value_counts()

3     531487
1     531019
0     530594
8     530453
7     530403
2     530290
11    529943
6     529904
5     529809
4     529762
10    529683
9     529273
Name: partition, dtype: int64

### Write Prepared

In [38]:
# Necesita paquete 'pyarrow'

In [39]:
df.to_parquet(path_out, partition_cols='partition', compression='snappy')

In [24]:
# Para leer el parquet:
p_df = pd.read_parquet(path_out).drop(columns=['partition'])

In [82]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [83]:
df

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [93]:
tipo='TRANSFER'

dc = {
        'step':5,
        'amount':5,
        'newbalanceOrig':5,
        'oldbalanceOrg':5,
        'oldbalanceDest':5,
        'newbalanceDest':5,
        'isFlaggedFraud':5,
        'CASH_IN':0,
        'CASH_OUT':0,
        'DEBIT':0,
        'PAYMENT':0,
        'TRANSFER':0,
    }

dc['diffOrigen'] = dc['newbalanceOrig'] - dc['oldbalanceOrg']
dc['diffDestino'] = dc['newbalanceDest'] - dc['oldbalanceDest']
dc['cambioOrigen'] = (dc['diffOrigen'] + 0.01)/(dc['oldbalanceOrg'] + 1e3)
dc['cambioDestino'] = (dc['diffDestino'] + 0.01)/(dc['oldbalanceDest'] + 1e3)

dc[tipo]=1

In [95]:
pd.DataFrame({"fila1":dc}).transpose()

,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,amount,cambioDestino,cambioOrigen,diffDestino,diffOrigen,isFlaggedFraud,newbalanceDest,newbalanceOrig,oldbalanceDest,oldbalanceOrg,step
fila1,0.0,0.0,0.0,0.0,1.0,5.0,0.00001,0.00001,0.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0


In [96]:
df=pd.DataFrame(index=range(1))
df['step']=1
df['amount']=11
df['newbalanceOrig']=1
df['oldbalanceOrg']=1
df['newbalanceDest']=1
df['oldbalanceDest']=1
df['isFlaggedFraud']=1
df['CASH_IN']=0
df['CASH_OUT']=0
df['DEBIT']=0
df['PAYMENT']=0
df['TRANSFER']=0
df['diffOrigen'] = df['newbalanceOrig'] - df['oldbalanceOrg']
df['diffDestino'] = df['newbalanceDest'] - df['oldbalanceDest']
df['cambioOrigen'] = (df['diffOrigen'] + 0.01)/(df['oldbalanceOrg'] + 1e3)
df['cambioDestino'] = (df['diffDestino'] + 0.01)/(df['oldbalanceDest'] + 1e3)

In [98]:
from joblib import load
model = load('../random_forest_sm_final.joblib')

In [108]:
model.predict(df)

array([0], dtype=int64)

In [109]:
from sklearn.calibration import CalibratedClassifierCV

In [110]:
nuevo = CalibratedClassifierCV(base_estimator=model, method='sigmoid')

In [113]:
model.predict_proba(df)[0]

array([0.71, 0.29])